
# BIOINFORMATIKA - TSP – Genetikus algoritmus
**Feladat:** Traveling Salesman Problem (TSP)  

*   **Reprezentáció:** permutáció (városok sorrendje)
*   **Fitness:** –úthossz (mert a GA maximál, mi minimalizálnánk)


**GA-lépések:**
*  Szelekció: tournament
*  Keresztezés: OX/PMX
*  Mutáció swap/inversion
*  Visszahelyezés: generációs + elitizmus

**Grafikonok:**
* tanulási görbe (legjobb és átlag út hossza generációnként),
* a legjobb útvonal rajza (városok pontként, kör útvonallal).


Gyakorló feladatok


## FELADAT (könnyű) - Távolságmátrix és útvonalhossz

Írj egy függvényt, amely:

bemenet:
* coords: lista (vagy NumPy tömb) városokkal, pl. [(x1, y1), (x2, y2), ...]

kimenet:
* dist_matrix: kétdimenziós lista vagy NumPy tömb, ahol dist_matrix[i][j] az i. és j. város távolsága (euklideszi)

In [ ]:
def tour_length(tour, dist_matrix):
    """
    tour: pl. [0, 3, 1, 2, 4] – városindex-permutáció
    dist_matrix: NxN távolságmátrix
    Vissza: körút teljes hossza (visszatérés az első városba is).
    """
    ...


## FELADAT (könnyű) - Véletlen permutáció és swap mutáció

Írj két függvényt:
* random_tour(n)
* swap_mutation(tour)

*   List item
*   List item



majd

Futtasd le a swap_mutation-t néhányszor, és ellenőrizd, hogy:
* a permutáció hossza nem változik
* minden város továbbra is pontosan egyszer szerepel

In [ ]:
def random_tour(n):
    """Adjon vissza egy véletlen permutációt 0..n-1 között."""
    ...

def swap_mutation(tour):
    """
    Válassz ki véletlenszerűen két pozíciót a tour-ban és cseréld meg őket.
    Visszatér: új (mutált) tour.
    """
    ...


## FELADAT (könnyű) - Inversion mutáció

Írj egy inversion_mutation függvényt:
* inversion_mutation(tour)

majd

Teszteld:
* rajzold ki (akár textben) az eredeti és mutált tour-t
* számold ki a tour hosszát mutáció előtt/után ugyanazzal a dist_matrix-szel

In [ ]:
def inversion_mutation(tour):
    """
    Válassz ki két indexet i < j.
    Fordítsd meg a tour i..j közti szakaszát.
    Példa: [0, 1, 2, 3, 4, 5], i=1, j=4 → [0, 4, 3, 2, 1, 5]
    """
    ...


## FELADAT (Közepes) - GA inicializáció és egyszerű futtatás

Készíts:
* init_population(pop_size, n_cities)
* fitness(tour, dist_matrix)

Majd

Majd futtass egy nagyon egyszerű GA-t:
* populáció generálása
* minden generációban:
  * csak mutáció (swap vagy inversion), pl. minden egyedet kis valószínűséggel
  * új populáció értékelése
* futtass 50–100 generációt
* rajzold ki: generációnkénti legjobb tour hossz (vagy fitness)

In [ ]:
def init_population(pop_size, n_cities):
    """pop_size darab véletlen permutációból álló kezdeti populáció."""
    ...

def fitness(tour, dist_matrix):
    """Lehet pl. -tour_length (minimálásból max-olás)."""
    ...


## FELADAT (Közepes) - Crossover egyedekre (OX/PMX)

Implementálj egy TSP-kompatibilis keresztezőt (pl. OX vagy PMX):
* ox_crossover(parent1, parent2)

Feladatok:
* generálj 2 szülőt
* készíts 2 utódot
* ellenőrizd, hogy mindkét utód érvényes permutáció (minden város egyszer szerepel)
* mérd a szülők és utódok tour hosszát, írd ki összehasonlításként

In [ ]:
def ox_crossover(parent1, parent2):
    """Order Crossover (OX) TSP permutációkra."""
    ...


## FELADAT (Közepes) - Swap vs. Inversion – összehasonlító kísérlet

Készíts kódot, amely:
* ugyanazzal a GA-val (populációméret, generációszám, szelekció, crossover)
* két külön futtatást végez:
   * (a) csak swap_mutation
   * (b) csak inversion_mutation
* minden esetben többször (pl. 10–20 run) lefuttatod
* összehasonlítod:
   * átlagos legjobb tour hossz
   * szórás

Írj rövid következtetést:
* Melyik mutáció „stabilabb”?
* Van-e különbség a konvergencia sebességében?

# FELADAT (Nehéz) - érzékenységi vizsgálat és hibrid mutáció

Készíts olyan GA-t, amely:
* egyszerre használ swap + inversion mutációt: pl. minden mutáció-lépésnél:
   * 50% eséllyel swap_mutation
   * 50% eséllyel inversion_mutation

Feladatok:
1. **Válaszd ki a paramétereket:**
  * populációméret
  * mutációs ráta
  * generációk száma

2. **Használj legalább 20 városból álló TSP példát**.
3. **Készíts kísérleti tervet három mutációs stratégiára**:
  * csak swap
  * csak inversion
  * kevert swap+inversion
4. **Minden konfigurációt futtasd le legalább 20x**.
5. **Gyűjtsd ki**:
  * átlagos legjobb tour hossz
  * legjobb elért tour
  * konvergenciagörbéket (átlagolt fitness / generáció)
6. **Írj rövid rövid elemzést**:
  * melyik stratégia adott jobb eredményt
  *  hogyan hat a mutáció típusa a keresés jellegére (finomhangolás vs. nagy ugrások)